In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0,4"
device0 = "cuda:0" if torch.cuda.is_available() else "cpu"
device4 = "cuda:1" if torch.cuda.is_available() else "cpu"
OCCUPY_CUDA_0 = torch.randn(10, 10, device=device0)
OCCUPY_CUDA_4 = torch.randn(10, 10, device=device4)